# 在 Bedrock AgentCore Runtime 中部署 MCP 服务器

本研讨会演示如何使用 Amazon Bedrock AgentCore Runtime 部署和使用模型上下文协议（MCP）服务器，为 AI 代理实现可扩展和安全的自定义工具部署。

## 概述

在本实验中，您将：
- 创建具有网络搜索功能的自定义 MCP 服务器
- 使用 Amazon Cognito 设置身份验证
- 将 MCP 服务器部署到 Bedrock AgentCore Runtime
- 使用 Strands Agents 测试已部署的服务器
- 了解清理程序

## 先决条件

开始本实验之前，请确保您具备：
- 已配置 AWS 凭证（IAM 角色或环境变量）
- 已安装所需的 Python 包
- 对 MCP 和 Bedrock AgentCore 概念有基本了解

如果您没有在假设 IAM 角色的环境中运行，请将您的 AWS 凭证设置为环境变量：

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

为 MCP 服务器开发、Strands Agents 和 Bedrock AgentCore SDK 安装所需的包：

In [ ]:
#%pip install -q ddgs mcp strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit 

## 什么是用于 MCP 的 Bedrock AgentCore Runtime？

Amazon Bedrock AgentCore Runtime 允许您将模型上下文协议（MCP）服务器部署为托管的可扩展服务。主要优势包括：

- **可扩展性**：根据需求自动扩展
- **安全性**：内置身份验证和授权
- **托管基础设施**：无需管理服务器或容器
- **集成**：与 Bedrock 服务无缝集成

MCP 服务器提供 AI 代理可以用来扩展其功能的工具和资源，例如网络搜索、数据库访问或自定义业务逻辑。

### 创建自定义 MCP 服务器

让我们创建一个使用 DuckDuckGo 提供网络搜索功能的简单 MCP 服务器。该服务器将部署到 AgentCore Runtime 以供可扩展使用。

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# Define a websearch tool
@mcp.tool()
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> list:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

In [ ]:
%%writefile requirements.txt
ddgs
mcp
bedrock-agentcore

## MCP 服务器的本地测试

在部署到 AgentCore Runtime 之前，先在本地测试 MCP 服务器。

### 步骤 1：启动 MCP 服务器

在终端中运行 MCP 服务器：

```bash
uv pip install -r requirements.txt
uv run mcp_server.py
```
or
```bash
pip install -r requirements.txt
python mcp_server.py
```

### 步骤 2：使用 Strands Agent 测试

执行以下代码来测试集成：

In [ ]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# Connect to the web search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"http://localhost:8000/mcp"
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available MCP tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with custom MCP tools
    agent = Agent(
        model="us.amazon.nova-pro-v1:0",  # Optional: Specify the model ID
        system_prompt="You are a helpful assistant that provides concise responses.",
        #system_prompt="""你是一位乐于助人的助手，提供简洁的答复。""",
        tools=mcp_tools,
    )

    #response = agent("Search for a coffee maker on amazon.com and extract details of the first one")
    response = agent("在 amazon.com 上搜索咖啡机并提取第一款咖啡机的详细信息")
    print(response) 

## 在 Bedrock AgentCore Runtime 中部署带身份验证的 MCP 服务器

现在我们将配置 MCP 服务器并将其部署到 Bedrock AgentCore Runtime。此过程涉及创建依赖项、配置身份验证和部署服务。
![bedrock-agentcore-runtime-launch](images/runtime-launch.png)

### 步骤 1：设置 Amazon Cognito 身份验证

为已部署的 MCP 服务器创建 Cognito 用户池以实现安全访问。

创建的组件
- **用户池**：管理用户身份
- **应用客户端**：启用应用程序身份验证
- **测试用户**：用于测试身份验证流程

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Initialize Cognito client
cognito_client = boto3.client('cognito-idp', region_name=region)


# Create User Pool
user_pool_response = cognito_client.create_user_pool(
    PoolName='MCPServerPool',
    Policies={
        'PasswordPolicy': {
            'MinimumLength': 8
        }
    }
)
cognito_pool_id = user_pool_response['UserPool']['Id']

# Create App Client
app_client_response = cognito_client.create_user_pool_client(
    UserPoolId=cognito_pool_id,
    ClientName='MCPServerPoolClient',
    GenerateSecret=False,
    ExplicitAuthFlows=[
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']

# Create User
cognito_client.admin_create_user(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    TemporaryPassword='Temp123!',
    MessageAction='SUPPRESS'
)

# Set Permanent Password
cognito_client.admin_set_user_password(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    Password='MyPassword123!',
    Permanent=True
)


# Output the required values
print(f"Pool id: {cognito_pool_id}")
print(f"Discovery URL: https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration")
print(f"Client ID: {cognito_client_id}")

### 步骤 2：配置 Bedrock AgentCore Runtime

Set up the Bedrock AgentCore Runtime configuration with automatic resource creation.

**Generated Artifacts:**
This step creates essential deployment files:
- **Dockerfile**: Container configuration for the MCP Server
- **.dockerignore**: list the excluded files when docker build
- **.bedrock_agentcore.yaml**: Runtime deployment configuration

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name="mcp_server_agentcore",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration",
        }
    }
)
print("Configuration completed ✓")

### 步骤 3：部署到 Bedrock AgentCore Runtime

使用 AWS CodeBuild 启动部署过程以进行容器化和部署。

**部署过程：**
- 构建 MCP 服务器的容器化版本
- 创建所需的 AWS 资源（ECR 存储库、IAM 角色）
- 将容器镜像推送到 Amazon ECR
- 作为托管的自动扩展服务部署到 AgentCore Runtime

In [ ]:
launch_result = agentcore_runtime.launch()

### 验证部署状态

检查部署状态并等待运行时准备就绪：

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

mcp_runtime_id = launch_result.agent_id
mcp_runtime_arn = launch_result.agent_arn
mcp_ecr_repo_name = launch_result.ecr_uri.split('/')[1]
print(f"MCP AgentCore Runtime ID: {mcp_runtime_id}")
print(f"MCP AgentCore Runtime ARN: {mcp_runtime_arn}")
print(f"ECR Repo for MCP AgentCore Runtime: {mcp_ecr_repo_name}")

### 使用 Strands Agent 测试已部署的 MCP 服务器作为工具

现在让我们通过 Bedrock AgentCore Runtime 端点连接到已部署的 MCP 服务器并进行适当的身份验证来测试它。

In [ ]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import boto3

# Get bearer token (access token) from Cognito Auth 
cognito_client = boto3.client('cognito-idp', region_name=boto3.session.Session().region_name)
auth_response = cognito_client.initiate_auth(
    ClientId=cognito_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']

mcp_runtime_arn = launch_result.agent_arn
encoded_arn = mcp_runtime_arn.replace(':', '%3A').replace('/', '%2F')

# Connect to the Web Search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    #"Content-Type": "application/json"
}
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url, headers))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with custom MCP tools
    agent = Agent(
        model="us.amazon.nova-pro-v1:0",  # Optional: Specify the model ID
        system_prompt="You are a helpful assistant that provides concise responses.",
        #system_prompt="""你是一位乐于助人的助手，提供简洁的答复。""",
        tools=mcp_tools,
    )

    #response = agent("Search for a coffee maker on amazon.com and extract details of the first one")
    response = agent("在 amazon.com 上搜索咖啡机并提取第一款咖啡机的详细信息")
    print(response) 

Let's examine the detailed execution flow of the agent loop to understand how the agent processes requests and generates responses:

In [ ]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

### 为实验 4 保存配置

存储部署参数以在下一个实验中使用：

In [ ]:
%store cognito_client_id
%store cognito_pool_id
%store mcp_runtime_arn
%store mcp_runtime_id
%store mcp_ecr_repo_name

## 资源清理（可选）

**⚠️ 如果继续进行实验 4，请跳过此部分**

清理已部署的资源：

In [ ]:
import boto3
import os

region = boto3.session.Session().region_name

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_client.delete_repository(repositoryName=launch_result.ecr_uri.split('/')[1], force=True)
    print("✓ ECR repository deleted")

    print("Deleting Cognito User Pool...")
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito User Pool deleted")

    print("Deleting Bedrock AgentCore configuration file...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml deleted")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")